In [ ]:
import os
import requests
import pandas as pd
import datetime as dt

BASE = "https://api.elections.kalshi.com/trade-api/v2"

def unix_ts(d: dt.datetime) -> int:
    return int(d.replace(tzinfo=dt.timezone.utc).timestamp())

def iter_markets(series_ticker, status="settled", min_close_ts=None, max_close_ts=None, limit=1000):
    cursor = ""
    while True:
        params = {"series_ticker": series_ticker, "status": status, "limit": limit}
        if cursor:
            params["cursor"] = cursor
        if min_close_ts is not None:
            params["min_close_ts"] = int(min_close_ts)
        if max_close_ts is not None:
            params["max_close_ts"] = int(max_close_ts)

        r = requests.get(f"{BASE}/markets", params=params, timeout=30)
        r.raise_for_status()
        data = r.json()

        for m in data.get("markets", []):
            yield m

        cursor = data.get("cursor") or ""
        if not cursor:
            break

# ---------- CONFIG YOU EDIT ----------
SERIES_TICKER = "KXNCAAWBGAME"
STATUS = "settled"

# Optional: filter by close time window (UTC). Set to None to fetch all settled in the series.
# Example window:
# start_ts = unix_ts(dt.datetime(2025, 12, 1, 0, 0, 0))
# end_ts   = unix_ts(dt.datetime(2026, 1, 1, 0, 0, 0))
start_ts = None
end_ts   = None

# Output path you will specify:
OUTPUT_CSV = r"/users/coop/desktop/w_hoops/kalshi/tickers.csv"
# -----------------------------------

rows = []
for m in iter_markets(SERIES_TICKER, status=STATUS, min_close_ts=start_ts, max_close_ts=end_ts):
    rows.append({
        "ticker": m.get("ticker"),
        "event_ticker": m.get("event_ticker"),
        "series_ticker": m.get("series_ticker"),
        "status": m.get("status"),
        "open_ts": m.get("open_time"),
        "close_ts": m.get("close_time"),
        "title": m.get("title"),
    })

df = pd.DataFrame(rows).dropna(subset=["ticker"]).drop_duplicates(subset=["ticker"])

# Ensure output directory exists
out_dir = os.path.dirname(OUTPUT_CSV)
if out_dir:
    os.makedirs(out_dir, exist_ok=True)

df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved {len(df):,} markets to: {OUTPUT_CSV}")

In [ ]:
import datetime as dt
from zoneinfo import ZoneInfo
import requests

BASE = "https://api.elections.kalshi.com/trade-api/v2"
TICKER = "KXNCAAWBGAME-25DEC21MERTULN-MER"   # <- your ticker

LOCAL_TZ = ZoneInfo("America/New_York")
TARGET_HOUR_LOCAL = 8
WINDOW_MINUTES = 30          # widen window while debugging
PERIOD_INTERVAL_MIN = 1      # 1-minute candles

HEADERS = {}  # add auth headers only if you get 401/403

def parse_iso_z(s: str) -> dt.datetime:
    return dt.datetime.fromisoformat(str(s).replace("Z", "+00:00"))

def dollars_close_or_prev(obj):
    if not obj:
        return None
    return obj.get("close_dollars") if obj.get("close_dollars") is not None else obj.get("previous_dollars")

def pick_snapshot_candle(candles, target_start_ts: int):
    # Prefer candle ending at 8:01 (covers 8:00–8:01)
    preferred_end = target_start_ts + 60
    candles = sorted(candles, key=lambda c: c.get("end_period_ts", 0))
    for c in candles:
        if c.get("end_period_ts") == preferred_end:
            return c
    for c in candles:
        if c.get("end_period_ts", 0) > target_start_ts:
            return c
    return None

# 1) Resolve/validate ticker via GET /markets?tickers=... (comma-separated list)  [oai_citation:3‡Kalshi API Documentation](https://docs.kalshi.com/api-reference/market/get-markets)
r = requests.get(
    f"{BASE}/markets",
    params={"tickers": TICKER},
    headers=HEADERS,
    timeout=30
)
r.raise_for_status()
j = r.json()
markets = j.get("markets", [])

if not markets:
    raise ValueError(
        f"No market returned for ticker={TICKER}. "
        "This usually means the ticker is wrong (typo / not canonical). "
        "Use the exact ticker from your saved settled-markets CSV."
    )

m = markets[0]
canonical_ticker = m["ticker"]
close_time_utc = parse_iso_z(m["close_time"])
game_date_local = close_time_utc.astimezone(LOCAL_TZ).date()

target_local = dt.datetime(game_date_local.year, game_date_local.month, game_date_local.day,
                           TARGET_HOUR_LOCAL, 0, 0, tzinfo=LOCAL_TZ)
target_utc = target_local.astimezone(dt.timezone.utc)

start_ts = int(target_utc.timestamp())
end_ts = start_ts + WINDOW_MINUTES * 60

print("Resolved ticker:", canonical_ticker)
print("Title:", m.get("title"))
print("Close time (UTC):", close_time_utc.isoformat())
print("Game date (local):", game_date_local)
print("Target (local):", target_local.isoformat())
print("Window (UTC):", dt.datetime.fromtimestamp(start_ts, tz=dt.timezone.utc).isoformat(),
      "->", dt.datetime.fromtimestamp(end_ts, tz=dt.timezone.utc).isoformat())

# 2) Candlesticks (batch endpoint; market_tickers is a comma-separated string)  [oai_citation:4‡Kalshi API Documentation](https://docs.kalshi.com/api-reference/market/batch-get-market-candlesticks)
cs = requests.get(
    f"{BASE}/markets/candlesticks",
    params={
        "market_tickers": canonical_ticker,   # string ok for 1 ticker
        "start_ts": start_ts,
        "end_ts": end_ts,
        "period_interval": PERIOD_INTERVAL_MIN,
        # leave this OFF while debugging; add back once working:
        # "include_latest_before_start": "true",
    },
    headers=HEADERS,
    timeout=30
)

if cs.status_code != 200:
    print("\nCandlesticks request failed.")
    print("URL:", cs.url)
    print("Status:", cs.status_code)
    print("Body:", cs.text[:2000])
    cs.raise_for_status()

csj = cs.json()
mkts = csj.get("markets", [])
candles = mkts[0].get("candlesticks", []) if mkts else []
print("\nCandles returned:", len(candles))

snap = pick_snapshot_candle(candles, start_ts)
if snap is None:
    print("No candle found after 8:00 in this window.")
else:
    end_period_ts = snap.get("end_period_ts")
    end_period_utc = dt.datetime.fromtimestamp(end_period_ts, tz=dt.timezone.utc).isoformat()

    price = dollars_close_or_prev(snap.get("price") or {})
    bid   = dollars_close_or_prev(snap.get("yes_bid") or {})
    ask   = dollars_close_or_prev(snap.get("yes_ask") or {})

    print("Snapshot candle end (UTC):", end_period_utc)
    print("price_close_dollars:", price)
    print("yes_bid_close_dollars:", bid)
    print("yes_ask_close_dollars:", ask)
    print("volume:", snap.get("volume"), "open_interest:", snap.get("open_interest"))